<a href="https://colab.research.google.com/github/dhanushasubbaraj/imlo/blob/main/imlo%20project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torchvision.transforms.functional import get_image_size
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score


train_transform = transforms.Compose([
    transforms.RandomRotation(degrees=(-30, 30)),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

training_data = datasets.Flowers102(
    root="train_data",
    split='train',
    download=True,
    transform=train_transform
)

test_data = datasets.Flowers102(
    root="test_data",
    split='test',
    download=True,
    transform=transform
)

validation_data = datasets.Flowers102(
    root="validation_data",
    split='val',
    download=True,
    transform=transform
)

train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print(torch.cuda.is_available())
torch.cuda.empty_cache()
print(torch.cuda.memory_allocated())

class My_NN(nn.Module):
    def __init__(self,):
        super().__init__()
        self.flatten = nn.Flatten()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=2, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=2, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=2, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(128 * 30 * 30, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Linear(2048,1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 102)
)

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.linear_relu_stack(x)

        return x

model = My_NN().to(device)
loss_fn = nn.CrossEntropyLoss()
weight_decay = 1e-6
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.0001, weight_decay=weight_decay)
epochs = 200
batch_size = 32

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, train_accuracy = 0, 0
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        optimizer.zero_grad()
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        train_accuracy += (pred.argmax(1) == y).type(torch.float).sum().item()

        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= num_batches
    train_accuracy /= size
    return train_accuracy * 100, train_loss


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, test_accuracy = 0, 0
    preds, targets = [], []
    model.eval()

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            test_accuracy += (pred.argmax(1) == y).type(torch.float).sum().item()
            preds.append(pred.argmax(1).cpu().numpy())
            targets.append(y.cpu().numpy())

    test_loss /= num_batches
    test_accuracy /= size

    preds = np.concatenate(preds)
    targets = np.concatenate(targets)

    f1 = f1_score(targets, preds, average='weighted')

    return test_accuracy * 100, test_loss, f1



best_loss = float('inf')
patience = 10
counter = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------------------------")
    train_accuracy, train_loss = train(train_dataloader, model, loss_fn, optimizer)
    print(f"Training Accuracy: {train_accuracy:.2f}%, Training Loss: {train_loss:.4f} \n")
    val_accuracy, val_loss, val_f1 = test(validation_dataloader, model, loss_fn)
    print(f"Validation Accuracy: {val_accuracy:.2f}%, Validation Loss: {val_loss:.4f}, Validation F1-score: {val_f1:.4f} \n")
    if val_loss < best_loss:
        best_loss = val_loss
        counter = 0

    else:
        counter += 1

    if counter >= patience:
        print(f'Validation loss has not improved for {patience} epochs. Early stopping.')
        break
print("Done")

test_accuracy, test_loss, test_f1 = test(test_dataloader, model, loss_fn)
print(f"Test Accuracy: {test_accuracy:.2f}%, Test Loss: {test_loss:.4f}, Test F1-score: {test_f1:.4f}")


Using device: cuda
True
4584619008
Epoch 1
-----------------------------------------
Training Accuracy: 3.33%, Training Loss: 4.5317 

Validation Accuracy: 2.45%, Validation Loss: 4.5890, Validation F1-score: 0.0064 

Epoch 2
-----------------------------------------
Training Accuracy: 13.24%, Training Loss: 4.2085 

Validation Accuracy: 9.22%, Validation Loss: 4.3134, Validation F1-score: 0.0684 

Epoch 3
-----------------------------------------
Training Accuracy: 22.75%, Training Loss: 4.0170 

Validation Accuracy: 13.53%, Validation Loss: 4.2346, Validation F1-score: 0.1010 

Epoch 4
-----------------------------------------
Training Accuracy: 28.43%, Training Loss: 3.9201 

Validation Accuracy: 14.80%, Validation Loss: 4.1848, Validation F1-score: 0.1172 

Epoch 5
-----------------------------------------
Training Accuracy: 35.59%, Training Loss: 3.8135 

Validation Accuracy: 16.47%, Validation Loss: 4.1615, Validation F1-score: 0.1329 

Epoch 6
-----------------------------------

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
